Drawing the corner from HDFBackend. You can apply different burnin to it, and you can get extra information from it.

In [2]:
from emcee.backends import HDFBackend
import numpy as np
import corner
import matplotlib.pyplot as plt
from read_files import *
from calc_wp import w_p
from run_mcmc import HODParameter

In [16]:
with open('config_small.yaml', 'r') as f:
    par_configs, other_configs = list(yaml.load_all(f, yaml.FullLoader))
# ========== reading configs ==========

# don't set the lgMmin too large or sig_lgM too small, otherwise N_c will be all zeros
available_fields = ['Nwalkers', 'Nstep', 'Nburnin', 'Npro', 'auto_range', 'cross_range',
                    'backend_file', 'numpy_file', 'wp_table_path', 'signal_path']
Nwalkers = 40
Nstep = 4000
Nburnin = 300
Npro = 40
auto_range = None
cross_range = None
backend_file = ''
numpy_file = ''
wp_table_path = '../wp_table'
signal_path = '../signal'

for k in available_fields:
    if k in other_configs:
        locals()[k] = other_configs[k]

wp_table = read_wp(wp_table_path)
rp_auto, signal_auto, rp_cross, signal_cross = read_signal(signal_path + '/signal.npy')
cov = read_cov(signal_path + '/cov.npy')
logM, Nh = read_halo_mass_function('../halo_mass_function.npy')
# for better performance I would recommend setting these as global variable

# ========== apply the fitting range ==========

if auto_range or cross_range:
    if auto_range is None:
        auto_range = [0, len(rp_auto)]
    if cross_range is None:
        cross_range = [0, len(rp_cross)]
    l_a, r_a = auto_range[0], auto_range[1]
    l_c, r_c = cross_range[0], cross_range[1]

    auto_size = len(rp_auto)
    cross_size = len(rp_cross)
    rp_auto = rp_auto[l_a:r_a]
    rp_cross = rp_cross[l_c:r_c]
    signal_auto = signal_auto[l_a:r_a]
    signal_cross = signal_cross[l_c:r_c]

    cov = np.vstack((
        np.hstack((cov[l_c:r_c, l_c:r_c], cov[l_c:r_c, cross_size+l_a:cross_size+r_a])),
        np.hstack((cov[cross_size+l_a:cross_size+r_a, l_c:r_c], cov[cross_size+l_a:cross_size+r_a, cross_size+l_a:cross_size+r_a]))
    ))

signal = np.hstack((signal_cross, signal_auto))
cov_inv = np.linalg.inv(cov)

In [18]:
wp_table_auto = interpolate_table(wp_table, rp_auto)

In [ ]:
# ========== handle interpolating table ==========
wp_table_auto = interpolate_table(wp_table, rp_auto)
wp_table_cross = interpolate_table(wp_table, rp_cross)

parameter = HODParameter.from_config(par_configs)

In [20]:
rp_auto, signal_auto, rp_cross, signal_cross = read_signal(signal_path + '/signal.npy')
cov = read_cov(signal_path + '/cov.npy')

In [22]:
len(rp_auto) - 2

25

In [21]:
cov.shape

(42, 42)

In [19]:
cov.shape

(13, 13)